**Másteres Universitarios en Ciencia de Datos, Ingeniería Informática, e Innovación en Inteligencia Computacional y Sistemas Interactivos, UAM**
## **Procesamiento del Lenguaje Natural**
# **Práctica de laboratorio 4: Ajuste fino de BERT para análisis de sentimientos**

---




En esta práctica, vamos a ajustar (hacer *fine-tuning* de) BERT para la tarea de análisis de sentimiento de la práctica 3.

Al hacerlo, estaremos posibilitando la transferencia de aprendizaje (*transfer learning*) de un modelo de lenguaje preentrenado, lo cual ha quedado demostrado ser una estrategia exitosa para lograr rendimientos de última generación en tareas de procesamiento del lenguaje natural.

Aprenderemos lo siguiente:

*   Ajustar modelos transformers de la biblioteca [Hugging Face](https://github.com/huggingface/transformers).
*   Preprocesar datos para la arquitectura de transformers (tokenización de subpalabras con WordPiece).
*   Implementar un clasificador basado en BERT.

Como aprendimos en la clase de teoría, utilizaremos el conocimiento codificado en el Transformer para mejorar el aprendizaje de nuestra tarea objetivo. Así, nuestro clasificador de sentimientos tendrá dos componentes principales:

*   El codificador de texto (encoder) BERT, que no sabe nada sobre sentimientos, pero sí sobre el idioma inglés.
*   Un componente dedicado a la clasificación de sentimientos, que será una capa feed-forward.

BERT generará los embeddings para las frases entrada y los pasará a la capa de clasificación. Cuando ajustemos el clasificador, también modificaremos los parámetros de BERT para que aprenda aspectos específicos de la tarea.

Ventajas de esta arquitectura y del aprendizaje por transferencia:

*   Se puede recopilar una cantidad ilimitada de texto no etiquetado desde la web con muy poco esfuerzo para entrenar un modelo de lenguaje a gran escala.
*   Transformer es una arquitectura feed-forward que permite un entrenamiento altamente paralelizado y eficiente en conjuntos de datos masivos, con el objetivo de predecir palabras basadas en su contexto ([consulta el tutorial sobre estrategias de aprendizaje para clasificación de secuencias](https://colab.research.google.com/drive/1yWaLpCWImXZE2fPV0ZYDdWWI8f52__9A#scrollTo=MGqVkG2-7qfu)).
*   Aunque el preentrenamiento de un modelo de lenguaje puede ser costoso, el *fine-tuning* generalmente puede realizarse en una sola GPU, ya que típicamente requiere pocas épocas de entrenamiento.

## 1. Carga de los datos
Usaremos los mismos datos de análisis de sentimiento que en la práctica anterior. Por lo tanto, primero necesitamos montar nuestra cuenta de Google Drive, y después acceder al Stanford Sentiment Treebank (SST).

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import re

import tensorflow as tf
from sklearn.utils import shuffle

## Fijamos semillas de generación de números aleatorios para replicabilidad de resultados
np.random.seed(1)
tf.random.set_seed(2)

# Utilizamos el Stanford Sentiment Treebank (SST) como corpus de prueba para análisis de sentimientos de textos
# Cargamos y procesamos los datos para tratarlos como un problema de clasificación binaria de sentimientos
# Convertimos la escala de sentimientos de [1,5] a {0,1}
def load_sst_data(path,
                  easy_label_map={0:0, 1:0, 2:None, 3:1, 4:1}):
    data = []
    with open(path) as f:
        for i, line in enumerate(f):
            example = {}
            example['label'] = easy_label_map[int(line[1])]
            if example['label'] is None:
                continue

            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)
    data = pd.DataFrame(data)
    return data

def pretty_print(example):
    print('Label: {}\nText: {}'.format(example['label'], example['text']))

sst_home = '/media/dong/三星disk1/Users/Xudon/Documents/GitHub/NLP/Lab_3/nlp2425-lab3_data/data/'
training_set = load_sst_data(sst_home+'/sst_training.txt')
validation_set = load_sst_data(sst_home+'/sst_validation.txt')
test_set = load_sst_data(sst_home+'/sst_test.txt')

# Desordenamos los datos
training_set = shuffle(training_set)
validation_set = shuffle(validation_set)
test_set = shuffle(test_set)

# Obtenemos los vectores de texto y etiquetas (clases de opinión: positiva y negativa)
train_texts = training_set.text
train_labels = training_set.label

validation_texts = validation_set.text
validation_labels = validation_set.label

test_texts = test_set.text
test_labels = test_set.label

print('Tamaño del conjunto de entrenamiento: {}'.format(len(training_set)))
print('Tamaño del conjunto de validación: {}'.format(len(validation_set)))
print('Tamaño del conjunto de test: {}'.format(len(test_set)))

## 2. Instalación y configuración de la librería Transformers
Para usar la librería Transformers, deberemos registrarnos en Hugging Face y obtener una API key, que llamaremos HF_TOKEN. Esta clave nos permitirá acceder a modelos preentrenados y otros recursos de Hugging Face.

Para guardar y activar HF_TOKEN en Google Colab:


1.   Accede a Hugging Face Tokens y genera un nuevo token de acceso con permisos de lectura.
2.   Para evitar compartir tu clave de acceso en el código Python, puedes añadir HF_TOKEN como un "secreto" en Google Colab. Para hacerlo, sigue estos pasos:
    *   En Google Colab, haz clic en el icono de su panel de control en la parte izquierda.
    *   Luego, selecciona la pestaña "Secretos".
    *   Haz clic en "Nuevo secreto" y en el campo "Nombre" escribe HF_TOKEN. En el campo "Valor", pega tu clave de Hugging Face que obtuviste en el paso 1.

Después de guardar el secreto, en el código podrías acceder al valor de HF_TOKEN como sigue:
```
import os
HF_TOKEN = os.getenv('HF_TOKEN')
from huggingface_hub import login
login(HF_TOKEN)
```

De esta manera, usarías la clave de manera segura y acceder a modelos y datasets de Hugging Face sin exponerla directamente en el código. Si has añadido HF_TOKEN en "Secretos" de Google Colab, no es necesario añadir las instrucciones anteriores en el código.

In [ ]:
!pip install transformers

Una vez que la librería Transformers está instalada, podemos utilizarla directamente creando objetos de las siguientes clases:

- La clase **tokenizer**: se encarga de convertir una secuencia de entrada en tensores de enteros, que son índices en el vocabulario de un modelo. La tokenización varía según el modelo, ya que cada modelo tiene su propio tokenizer.

- La clase **model**: contiene la lógica del modelo de red neuronal en sí. Al utilizar un modelo de TensorFlow, hereda de tf.keras.layers.Layer, lo que significa que se puede usar de manera muy sencilla con la API fit de Keras o para hacer cosas más complicadas.

También existe una clase de configuración (**configuration**), que es necesaria a menos que no se estén utilizando los valores predeterminados. Con esta clase indicamos todo lo relacionado con los hiperparámetros del modelo, como el número de capas, dropout, etc. A continuación, se muestra un ejemplo de un archivo de configuración de BERT, para los pesos preentrenados bert-base-cased.

```
{
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}
```


Específicamente, importamos las clases `TFBertForSequenceClassification` y `BertTokenizer` desde la librería transformers. Utilizamos el método `from_pretrained` para cargar el modelo BERT preentrenado, en este caso el modelo "bert-base-uncased", que es una versión de BERT entrenada sin distinción de mayúsculas y minúsculas.


*   La variable `model` contiene el modelo BERT adaptado para tareas de clasificación de secuencias, que es el caso que nos interesa aquí pues buscamos clasificar de forma binaria frases atendiendo a sus polaridades de sentimiento, positiva o negativa.
*   Por su parte, el `tokenizer` es el que fue empleado para construir BERT, y el que utilizaremos para convertir el texto de entrada en tensores de índices que el modelo puede procesar.

In [ ]:
from transformers import TFBertForSequenceClassification, BertTokenizer
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

⬛
####EJERCICIO 1
Atendiendo a lo aprendido en la clase de teoría y a información disponible en la web, ¿cuál es la arquitectura interna de `TFBertForSequenceClassification` con respecto a la de `TFBertModel`?

[RESPONDER AQUÍ]

## 3. Preprocesado de los textos
A continuación, definimos dos funciones auxiliares para 1) extraer características del tokenizer (`convert_examples_to_features`) y 2) convertir las características en un objeto de la clase `tf.data.Dataset `(`convert_features_to_tf_dataset`).

`tf.data.Dataset` ayuda a gestionar e iterar de manera eficiente los datos de entrada y salida del modelo. Para más información, puedes consultar la API en la página web de TensorFlow: https://www.tensorflow.org/api_docs/python/tf/data/Dataset.

Con esas funciones preprocesamos los conjuntos de entrenamiento y validación. Usamos `tf.data.Dataset` para establecer el tamaño de lote en 32.

In [ ]:
from transformers import InputFeatures

def convert_examples_to_features(texts, labels):
  labels = list(labels)
  batch_encoding = tokenizer.batch_encode_plus(texts, max_length=128, pad_to_max_length=True)

  features = []
  for i in range(len(texts)):
      inputs = {k: batch_encoding[k][i] for k in batch_encoding}

      feature = InputFeatures(**inputs, label=labels[i])
      features.append(feature)

  for i, example in enumerate(texts[:5]):
      print("*** Ejemplo ***")
      print("texto: %s" % (example))
      print("características: %s" % features[i])

  return features

def convert_features_to_tf_dataset(features):
  def gen():
      for ex in features:
          yield (
              {
                  "input_ids": ex.input_ids,
                  "attention_mask": ex.attention_mask,
                  "token_type_ids": ex.token_type_ids,
              },
              ex.label,
          )
  dataset = tf.data.Dataset.from_generator(gen,
                                           ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
                                           (
                                               {
                                                "input_ids": tf.TensorShape([None]),
                                                "attention_mask": tf.TensorShape([None]),
                                                "token_type_ids": tf.TensorShape([None])
                                                },
                                            tf.TensorShape([]),
                                            ))
  return dataset

train_features = convert_examples_to_features(train_texts, train_labels)
train_dataset = convert_features_to_tf_dataset(train_features)

validation_features = convert_examples_to_features(validation_texts, validation_labels)
validation_dataset = convert_features_to_tf_dataset(validation_features)

train_dataset = train_dataset.shuffle(100).batch(32)
validation_dataset = validation_dataset.batch(32)

# Visualizamos un bacth de 32 ejemplos
#instance = list(train_dataset.take(1).as_numpy_iterator())
#print(instance)

⬛
####EJERCICIO 2
¿Qué es lo que hace y retorna como salida la función `tokenizer.batch_encode_plus`?

[RESPONDER AQUÍ]

## 4. Entendiendo el tokenizer

Cuando preprocesamos el texto de entrada para ser introducido en un encoder como BERT, típicamente seguimos tres pasos:

1.   Dividir las palabras en tokens (subwords).
2.   Agregar los tokens especiales como [CLS] y [SEP]. Estos tokens especiales ya están incluidos en el vocabulario del modelo, por lo que tienen su propio identificador de token.
3.   Sustituir los tokens por sus identificadores correspondientes. Después de este paso, obtenemos la forma adecuada para BERT.

El siguiente código muestra los resultados de estos tres pasos.



In [ ]:
sentence1 = "A visually stunning rumination on love."
sentence2 = "There ought to be a directing license, so that Ed Burns can have his revoked."

print('0. INPUT SENTENCE: {}'.format(sentence1))

# Tokenizamos una frase
sentence1_tokenized = tokenizer.tokenize(sentence1)
print('1. TOKENIZED SENTENCE: {}'.format(sentence1_tokenized))

# Añadimos tokens especiales
sentence1_tokenized_with_special_tokens = ['[CLS]'] + sentence1_tokenized + ['[SEP]']
print('2. ADD [CLS], [SEP]: {}'.format(sentence1_tokenized_with_special_tokens))

# Convertimoa los tokens a ids
sentence1_ids = tokenizer.convert_tokens_to_ids(sentence1_tokenized_with_special_tokens)
print('3. SENTENCE IDS: {}'.format(sentence1_ids))

⬛
####EJERCICIO 3
¿Qué le ocurrió a "rumination" después de la tokenización? ¿Por qué?
¿Cuáles son los identificadores de los tokens [CLS] y [SEP]?
¿De dónde vienen los identificadores de token?

[RESPONDER AQUÍ]

Los tres pasos se pueden realizar con las funciones `encode` o `batch_encode_plus`.

In [ ]:
sentence1_ids = tokenizer.encode(sentence1, add_special_tokens=True)
print('SENTENCE IDS: {}'.format(sentence1_ids))

batch_encoding = tokenizer.batch_encode_plus(
        [sentence1], max_length=128, pad_to_max_length=True,
    )
print('ENCODE PLUS: {}'.format(batch_encoding))

⬛
####EJERCICIO 4
¿Qué diferencias hay entre las funciones `tokenizer.encode` y `tokenizer.batch_encode_plus`?

[RESPONDER AQUÍ]

### Dos frases como entrada

Como hemos visto en la clase de teoría, BERT es un modelo de lenguaje enmascarado que aprende prediciendo palabras ocultas y, además, determina si una segunda frase sigue a una primera. Por esta razón, el tokenizer de BERT está diseñado para aceptar dos oraciones como entrada. Este tipo de preprocesamiento de datos es especialmente útil para tareas como similitud textual semántica (*semantic textual similarit*y), inferencia en lenguaje natural (*natural language inference*, *textual entailment*), o búsqueda de respuesta (*question answering*).

In [ ]:
sentence_pair_ids = tokenizer.encode(text=sentence1, text_pair=sentence2, add_special_tokens=True)
batch_encoding = tokenizer.batch_encode_plus(
        [(sentence1, sentence2)], max_length=128, pad_to_max_length=True,
    )

print("SENTENCE PAIR IDS: {}".format(sentence_pair_ids))

⬛
####EJERCICIO 5
¿Qué IDs corresponden a la primera frase y cuáles a la segunda? ¿Cómo quedan separados?



[RESPONDER AQUÍ]

## 5. Ajuste fino (fine-tuning) de BERT como un clasificador de frases

Una característica interesante al ajustar un modelo grande es que no necesitamos entrenar durante muchas épocas.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history = model.fit(train_dataset, epochs=3, validation_data=validation_dataset)

In [ ]:
import matplotlib.pyplot as plt

# Visiualizamos el loss sobre los conjuntos de entrenamiento y validación en las 3 épocas de entranmiento
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

# Visiualizamos el accuracy sobre los conjuntos de entrenamiento y validación en las 3 épocas de entranmiento
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

⬛
####EJERCICIO 6
Una vez que el modelo esté ajustado para el análisis de sentimientos, podemos evaluarlo en el conjunto de test. Para ello, también necesitamos tokenizar la entrada y convertirla a IDs. Escribe a continuación el código correspondiente.

In [ ]:
# Recordatorio del modelo y tokenizer cargados arriba
#model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# AÑADIR CÓDIGO AQUÍ

⬛
####EJERCICIO 7
Considerar el modelo `TFBertModel` en vez de `TFBertForSequenceClassification`.

Modifica la arquitectura de `TFBertModel` para replicar los resultados obtenidos por `TFBertForSequenceClassification` en el ejercicio 5.

Para ello, tienes que añadir una capa feed-forward para clasificación binaria, que reciba como entrada la representación del token [CLS] generada por `TFBertModel`. Además, debes especificar las entadas `input_ids`, `attention_mask` y `token_type_ids`, así como decidir si añades `dropout` al clasificador, y si usas alternativas a `optimizer` y `loss`.

In [ ]:
from transformers import TFBertModel

# AÑADIR CÓDIGO AQUÍ

bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# AÑADIR CÓDIGO AQUÍ

history = model.fit(train_dataset, epochs=3, validation_data=validation_dataset)

In [ ]:
# Visiualizamos el loss sobre los conjuntos de entrenamiento y validación en las 3 épocas de entranmiento
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

# Visiualizamos el accuracy sobre los conjuntos de entrenamiento y validación en las 3 épocas de entranmiento
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()